In [1]:
%pip install fireducks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.5 MB/s eta 0:00:00


In [2]:
import cv2
import os
# import fireducks.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from google.colab import files

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"subrahmanya090","key":"9701e2a8a31cfdde3176e5e81626d2cf"}'}

In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d subrahmanya090/arecanut -p /content/areca-nut

Dataset URL: https://www.kaggle.com/datasets/subrahmanya090/arecanut
License(s): apache-2.0
100% 4.68G/4.68G [00:53<00:00, 104MB/s] 
100% 4.68G/4.68G [00:53<00:00, 93.5MB/s]


In [7]:
!unzip /content/areca-nut/arecanut.zip -d /content/areca-nut/

Archive:  /content/areca-nut/arecanut.zip
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191315575.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191322886.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191828631.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191834963.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191846444.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191852083.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191858580.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191904151.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191909850.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191921179.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191939327.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191952589.jpg  
  inflating: /content/areca-nut/arecanut/Bette

In [3]:
def preprocess_images(input_folder, output_folder):

    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            img = cv2.imread(input_path)

            if img is None:
                print(f"Skipping {filename}, unable to read the file.")
                continue

            resized_img = cv2.resize(img, (2048, 2048))

            gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
            blurred_img = cv2.GaussianBlur(gray_img, (5, 5), sigmaX=1.0, sigmaY=1.0)
            
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            clahe_img = clahe.apply(gray_img)

                                                                            # blurred_img = cv2.GaussianBlur(gray_img, (9, 9), 0)

                                                                            # source for sharpening images: https://medium.com/@sajjadhadi/mastering-opencv2-in-15-days-day-3-image-filtering-and-enhancement-ae1095f09aa5

            sharpening_kernel = np.array([[-1, -1, -1],
                             [-1, 9, -1],
                             [-1, -1, -1]])
            sharpened_image = cv2.filter2D(clahe_img, -1, sharpening_kernel)
            
            sobelx = cv2.Sobel(blurred_img, cv2.CV_64F, 1, 0, ksize=3)  # Gradient in X direction
            sobely = cv2.Sobel(blurred_img, cv2.CV_64F, 0, 1, ksize=3)  # Gradient in Y direction
            
            gradient_magnitude = np.hypot(sobelx, sobely)
            gradient_magnitude = np.uint8(255 * gradient_magnitude / np.max(gradient_magnitude))
            

                                                                            # source for CLAHE (Contrast Limited Adaptive Histogram Equalization): https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html

            normalized_img = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX)
            normalized_img = np.uint8(normalized_img)


            cv2.imwrite(output_path, normalized_img)

    print("Preprocessing complete. Images saved to:", output_folder)

#  Step1: 1stchali images preprocessing
output_folder='preprocessed/'
input_folder='input/'

#  Step2: Gotu images preprocessing
#  output_folder='gotupreprocessed'
#  input_folder='/content/areca-nut/arecanut/Gotu'

#  Step2: Gotu images preprocessing
#  output_folder='kolepreprocessed'
#  input_folder='/content/areca-nut/arecanut/Kole'

preprocess_images(input_folder,output_folder)

Preprocessing complete. Images saved to: preprocessed/


In [23]:
#Step1: 1stchalipreprocessed images to zip
!zip -r 1stchali-preprocessed_images.zip /content/1stchalipreprocessed

#Step2: gotupreprocessed images to zip
!zip -r gotu-preprocessed_images.zip /content/gotupreprocessed

#Step3: kolepreprocessed images to zip
!zip -r kole-preprocessed_images.zip /content/kolepreprocessed

  adding: content/kolepreprocessed/ (stored 0%)
  adding: content/kolepreprocessed/IMG_20241102_191004414.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_181715574.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_181553081.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_190622252.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_173852733.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_184217998.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_182709447.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_174830992.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_182517352.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_185907619.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_173129389.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_175657403.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_2

In [18]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!cp stchali-preprocessed_images.zip /content/drive/MyDrive/
!cp gotu-preprocessed_images.zip /content/drive/MyDrive/
!cp kole-preprocessed_images.zip /content/drive/MyDrive/